In [380]:
import pandas as pd 
import numpy as np 
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
import pandas as pd
from numpy import dot, exp
import random
from sklearn.model_selection import KFold
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.metrics import log_loss
from sklearn.linear_model import LogisticRegression
import copy
import os

#kaggle read csv

In [381]:
# train = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/train.csv')
# test = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/test.csv')
# greeks = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/greeks.csv')

#local read csv

In [382]:
filepath = '../data/'
train = pd.read_csv(os.path.join(filepath, 'train.csv'), index_col='Id')
test = pd.read_csv(os.path.join(filepath, 'test.csv'), index_col="Id")
greeks = pd.read_csv(os.path.join(filepath, 'greeks.csv'), index_col="Id")

# quick data peek

In [383]:
train.head(5)


,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,...,FL,FR,FS,GB,GE,GF,GH,GI,GL,Class
Id,,,,,,,,,,,,,,,,,,,,,
000ff2bfdfe9,0.209377,3109.03329,85.200147,22.394407,8.138688,0.699861,0.025578,9.812214,5.555634,4126.58731,...,7.298162,1.73855,0.094822,11.339138,72.611063,2003.810319,22.136229,69.834944,0.120343,1
007255e47698,0.145282,978.76416,85.200147,36.968889,8.138688,3.632190,0.025578,13.517790,1.229900,5496.92824,...,0.173229,0.49706,0.568932,9.292698,72.611063,27981.562750,29.135430,32.131996,21.978000,0
013f2bd269f5,0.470030,2635.10654,85.200147,32.360553,8.138688,6.732840,0.025578,12.824570,1.229900,5135.78024,...,7.709560,0.97556,1.198821,37.077772,88.609437,13676.957810,28.022851,35.192676,0.196941,0
043ac50845d5,0.252107,3819.65177,120.201618,77.112203,8.138688,3.685344,0.025578,11.053708,1.229900,4169.67738,...,6.122162,0.49706,0.284466,18.529584,82.416803,2094.262452,39.948656,90.493248,0.155829,0
044fb8a146ec,0.380297,3733.04844,85.200147,14.103738,8.138688,3.942255,0.054810,3.396778,102.151980,5728.73412,...,8.153058,48.50134,0.121914,16.408728,146.109943,8524.370502,45.381316,36.262628,0.096614,1


In [384]:
test.head(5)


,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,...,FI,FL,FR,FS,GB,GE,GF,GH,GI,GL
Id,,,,,,,,,,,,,,,,,,,,,
00eed32682bb,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
010ebe33f668,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
02fa521e1838,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
040e15f562a2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
046e85c7cc7f,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [385]:
greeks.head(5)

,Alpha,Beta,Gamma,Delta,Epsilon
Id,,,,,
000ff2bfdfe9,B,C,G,D,3/19/2019
007255e47698,A,C,M,B,Unknown
013f2bd269f5,A,C,M,B,Unknown
043ac50845d5,A,C,M,B,Unknown
044fb8a146ec,D,B,F,B,3/25/2020


# greeks used in stratified k-fold strategy
reference: https://www.kaggle.com/code/tatudoug/logistic-regression-baseline

In [386]:
# sum greeks, put into one column
greeks['k'] = greeks['Alpha']+greeks['Beta']+greeks['Gamma']+greeks['Delta']
greeks = greeks.reset_index()
train = pd.merge(greeks[['k','Id']],train,on='Id')
#set Id as index
train = train.set_index('Id')
#quick check after merge and set index
display(train.head(5))
# name columns
names = ['AB', 'AF', 'AH', 'AM', 'AR', 'AX', 'AY', 'AZ', 'BC', 'BD ', 'BN',
       'BP', 'BQ', 'BR', 'BZ', 'CB', 'CC', 'CD ', 'CF', 'CH', 'CL', 'CR', 'CS',
       'CU', 'CW ', 'DA', 'DE', 'DF', 'DH', 'DI', 'DL', 'DN', 'DU', 'DV', 'DY',
       'EB', 'EE', 'EG', 'EH', 'EJ', 'EL', 'EP', 'EU', 'FC', 'FD ', 'FE', 'FI',
       'FL', 'FR', 'FS', 'GB', 'GE', 'GF', 'GH', 'GI', 'GL']
target_name = 'Class'

,k,AB,AF,AH,AM,AR,AX,AY,AZ,BC,...,FL,FR,FS,GB,GE,GF,GH,GI,GL,Class
Id,,,,,,,,,,,,,,,,,,,,,
000ff2bfdfe9,BCGD,0.209377,3109.03329,85.200147,22.394407,8.138688,0.699861,0.025578,9.812214,5.555634,...,7.298162,1.73855,0.094822,11.339138,72.611063,2003.810319,22.136229,69.834944,0.120343,1
007255e47698,ACMB,0.145282,978.76416,85.200147,36.968889,8.138688,3.632190,0.025578,13.517790,1.229900,...,0.173229,0.49706,0.568932,9.292698,72.611063,27981.562750,29.135430,32.131996,21.978000,0
013f2bd269f5,ACMB,0.470030,2635.10654,85.200147,32.360553,8.138688,6.732840,0.025578,12.824570,1.229900,...,7.709560,0.97556,1.198821,37.077772,88.609437,13676.957810,28.022851,35.192676,0.196941,0
043ac50845d5,ACMB,0.252107,3819.65177,120.201618,77.112203,8.138688,3.685344,0.025578,11.053708,1.229900,...,6.122162,0.49706,0.284466,18.529584,82.416803,2094.262452,39.948656,90.493248,0.155829,0
044fb8a146ec,DBFB,0.380297,3733.04844,85.200147,14.103738,8.138688,3.942255,0.054810,3.396778,102.151980,...,8.153058,48.50134,0.121914,16.408728,146.109943,8524.370502,45.381316,36.262628,0.096614,1


# data cleanup

In [387]:
# encode features
train['EJ'] = pd.Series(np.where(train.EJ.values == 'A', 1, 0),
          train.index)
test['EJ'] = pd.Series(np.where(test.EJ.values == 'A', 1, 0),
          test.index)
# fill nan data with mean values 
train[names] = train[names].fillna(train[names].mean())
test[names] = test[names].fillna(train[names].mean())
# clip values to avoid different values in the test set from train
test = test[names].clip(train[names].min(axis=0).values,train[names].max(axis=0).values, axis=1)
# data scaled to allow the features interaction (by multiplication)
scaler = StandardScaler()
train2 = copy.copy(train)
teste2 = copy.copy(test)
vals = scaler.fit_transform(train[names])
vals_test = scaler.transform(test[names])
train2[names] = vals
teste2[names] = vals_test

#what does the scaled data look like?

In [388]:
display(train2.head(20))

,k,AB,AF,AH,AM,AR,AX,AY,AZ,BC,...,FL,FR,FS,GB,GE,GF,GH,GI,GL,Class
Id,,,,,,,,,,,,,,,,,,,,,
000ff2bfdfe9,BCGD,-0.572153,-0.170975,-0.261669,-0.237889,-0.189295,-1.900558,-0.083417,-0.173502,-0.038354,...,0.162487,-0.035806,-0.250869,-0.940094,-0.410260,-0.655511,-0.948991,0.531241,-0.815752,1
007255e47698,ACMB,-0.709105,-1.097801,-0.261669,-0.028701,-0.189295,-0.750457,-0.083417,0.678919,-0.104787,...,-0.458281,-0.060566,0.113218,-1.145070,-0.410260,0.687893,-0.238862,-0.509218,1.304239,0
013f2bd269f5,ACMB,-0.015212,-0.377169,-0.261669,-0.094845,-0.189295,0.465662,-0.083417,0.519453,-0.104787,...,0.198331,-0.051023,0.596934,1.637944,-0.299210,-0.051850,-0.351743,-0.424754,-0.808323,0
043ac50845d5,ACMB,-0.480851,0.138196,0.012347,0.547477,-0.189295,-0.729610,-0.083417,0.112088,-0.104787,...,0.060027,-0.060566,-0.105234,-0.219883,-0.342195,-0.650833,0.858232,1.101332,-0.812311,0
044fb8a146ec,DBFB,-0.206946,0.100517,-0.261669,-0.356885,-0.189295,-0.628845,-0.013229,-1.649292,1.445139,...,0.236971,0.896815,-0.230064,-0.432313,0.099920,-0.318309,1.409422,-0.395228,-0.818054,1
04517a3c90bd,ACMB,-0.572153,-0.385562,-0.261669,-0.436720,-0.189295,-0.601048,-0.083417,0.455666,-0.104787,...,-0.458281,-0.060566,0.570928,0.119259,-0.410260,0.491176,-0.300777,0.881515,1.304239,0
049232ca8356,ACMB,-0.275422,-0.769368,-0.261669,-0.439076,0.493251,-1.424535,-0.083417,-0.924444,-0.104787,...,-0.089279,-0.053302,0.035199,-0.285103,0.421603,-0.586801,0.271554,1.366078,-0.821111,0
057287f2da6d,ACMB,-0.444330,-1.103156,-0.261669,-0.255403,-0.189295,-0.218839,-0.083417,-0.266282,-0.104787,...,0.100953,-0.060566,-0.110436,-0.228269,-0.152855,0.346076,0.377262,-0.528736,-0.818417,0
0594b00fb30a,ACMB,-0.279987,-0.114675,-0.261669,-0.144675,-0.189295,-0.597574,-0.083417,-0.534476,-0.067982,...,-0.058454,-0.046893,-0.271674,-0.348459,0.107616,-0.521789,-0.174305,0.112900,-0.820304,0


# Feature Engineering

# 2 order interactions:

mutliplying two together

In [389]:
def mult(df,nome1,nome2):
    a  = df[nome1]*df[nome2]
    return(a/max(a))

def sqrt_exp(df,n1,n2):
    a = np.sqrt(df[n1]**df[n2])
    return(a/max(a))

def div(df,nome1,nome2):
    a  = df[nome1]/df[nome2]
    if max(a) == 0:
        print("bad!")
    return(a/max(a))

h = []
ht = []

n = 1
for n1 in names:
    for n2 in names[n:]:
        h.append(mult(train2,n1,n2).rename(n1+'_mul_'+n2))
        h.append(sqrt_exp(train2,n1,n2).rename(n1+'_sqrt_exp_'+n2))
        # h.append(div(train2,n1,n2).rename(n1+'_div_'+n2))

        ht.append(mult(teste2,n1,n2).rename(n1+'_mul_'+n2))
        ht.append(sqrt_exp(teste2,n1,n2).rename(n1+'_sqrt_exp_'+n2))
        # ht.append(div(teste2,n1,n2).rename(n1+'_div_'+n2))

    n+=1
    
newF = pd.DataFrame(h)
newF_test = pd.DataFrame(ht)

display(newF)
display(newF_test)

Id,000ff2bfdfe9,007255e47698,013f2bd269f5,043ac50845d5,044fb8a146ec,04517a3c90bd,049232ca8356,057287f2da6d,0594b00fb30a,05f2bc0155cd,...,fb786fb02a65,fbb79ba9d642,fbc241daef00,fbd12c4ae88b,fd1dd68d51b4,fd3dafe738fd,fd895603f071,fd8ef6377f76,fe1942975e40,ffcca4ded3bb
AB_mul_AF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AB_log_mult_AF,0.052114,-0.001695,0.215673,0.076576,0.191251,0.028120,0.017865,-0.004208,0.145687,0.018349,...,0.027214,0.086089,0.066977,0.087318,0.021847,0.034330,0.020433,0.093519,0.001974,0.237414
AB_mul_AH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AB_log_mult_AH,0.029798,0.018346,0.223383,0.128082,0.084071,0.029798,0.068816,0.043292,0.067939,0.059878,...,0.115129,0.057439,0.077438,0.080282,0.023386,0.045625,0.129555,0.214524,0.075398,-0.212192
AB_mul_AM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GH_log_mult_GI,0.002984,0.087074,0.080615,-0.001330,-0.028706,0.013652,-0.036642,0.055976,0.343358,0.002492,...,0.035082,-0.102711,0.004559,-0.073118,0.042923,0.021859,-0.108407,-0.035724,0.039001,0.019051
GH_mul_GL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GH_log_mult_GL,0.000000,-0.000000,0.000000,0.000000,-0.000000,-0.000000,0.000000,0.000000,0.000000,-0.000000,...,-0.000000,-0.000000,0.000000,-0.000000,-0.000000,-0.000000,0.000000,-0.000000,0.000000,0.000000
GI_mul_GL,0.029886,NaN,NaN,0.016494,NaN,0.015133,0.013808,NaN,0.106770,NaN,...,NaN,NaN,NaN,0.052912,NaN,0.007479,0.009956,0.041301,NaN,0.037656


Id,00eed32682bb,010ebe33f668,02fa521e1838,040e15f562a2,046e85c7cc7f
AB_mul_AF,NaN,NaN,NaN,NaN,NaN
AB_log_mult_AF,-1.0,-1.0,-1.0,-1.0,-1.0
AB_mul_AH,NaN,NaN,NaN,NaN,NaN
AB_log_mult_AH,1.0,1.0,1.0,1.0,1.0
AB_mul_AM,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
GH_log_mult_GI,1.0,1.0,1.0,1.0,1.0
GH_mul_GL,NaN,NaN,NaN,NaN,NaN
GH_log_mult_GL,-1.0,-1.0,-1.0,-1.0,-1.0
GI_mul_GL,NaN,NaN,NaN,NaN,NaN


Information Value (IV) and Weights of evidence (WoE) are widely used in credit score models with logistic regression. IV is a bivariate metric which ranks the predictive power of the the variables in relation to the target. WoE will transform the data to a monotonic pattern which is fundamental to the Logistic Regression model to perform well.

In [390]:
#https://lucastiagooliveira.github.io/datascience/iv/woe/python/2020/12/15/iv_woe.html
def iv_woe(data, target, bins=10, show_woe=False):
    
    #Empty Dataframe
    newDF,woeDF = pd.DataFrame(), pd.DataFrame()
    
    #Extract Column Names
    cols = data.columns
    
    #Run WOE and IV on all the independent variables
    for ivars in cols[~cols.isin([target])]:
        if (data[ivars].dtype.kind in 'bifc') and (len(np.unique(data[ivars]))>10):
            binned_x = pd.qcut(data[ivars], bins,  duplicates='drop')
            d0 = pd.DataFrame({'x': binned_x, 'y': data[target]})
        else:
            d0 = pd.DataFrame({'x': data[ivars], 'y': data[target]})

        
        # Calculate the number of events in each group (bin)
        d = d0.groupby("x", as_index=False).agg({"y": ["count", "sum"]})
        d.columns = ['Cutoff', 'N', 'Events']
        
        # Calculate % of events in each group.
        d['% of Events'] = np.maximum(d['Events'], 0.5) / d['Events'].sum()

        # Calculate the non events in each group.
        d['Non-Events'] = d['N'] - d['Events']
        # Calculate % of non events in each group.
        d['% of Non-Events'] = np.maximum(d['Non-Events'], 0.5) / d['Non-Events'].sum()

        # Calculate WOE by taking natural log of division of % of non-events and % of events
        d['WoE'] = np.log(d['% of Events']/d['% of Non-Events'])
        d['IV'] = d['WoE'] * (d['% of Events'] - d['% of Non-Events'])
        d.insert(loc=0, column='Variable', value=ivars)
        #print("Information value of " + ivars + " is " + str(round(d['IV'].sum(),6)))
        temp =pd.DataFrame({"Variable" : [ivars], "IV" : [d['IV'].sum()]}, columns = ["Variable", "IV"])
        newDF=pd.concat([newDF,temp], axis=0)
        woeDF=pd.concat([woeDF,d], axis=0)

        #Show WOE Table
        if show_woe == True:
            print(d)
    return newDF, woeDF

In [391]:
a,b = iv_woe(train2, target_name, bins=10, show_woe=False)

In [392]:
# most important features based on IV
a.sort_values(by='IV',ascending=False).Variable.values 

array(['k', 'DU', 'GL', 'FL', 'CR', 'DA', 'AF', 'AB', 'BQ', 'DI', 'EB',
       'FD ', 'EE', 'EH', 'FR', 'CD ', 'DE', 'CC', 'BN', 'FI', 'FE', 'DH',
       'EU', 'GF', 'DF', 'BC', 'DL', 'AM', 'BP', 'AH', 'AR', 'GH', 'DN',
       'CS', 'GB', 'DY', 'CF', 'CB', 'GI', 'BD ', 'FC', 'BR', 'CU', 'FS',
       'AZ', 'EL', 'EJ', 'CW ', 'AX', 'GE', 'AY', 'EG', 'EP', 'CH', 'CL',
       'BZ', 'DV'], dtype=object)

In [393]:
# Reordering the dataframe to keep IV with higger values in front
trainE = train[a.sort_values(by='IV',ascending=False).Variable.values]
trainE[target_name] = train[target_name]
testeE = test[a.sort_values(by='IV',ascending=False).Variable.values[2:]]

# join the original vars and the interactions between them
ff = pd.concat([trainE,newF.T],axis=1)
ff_teste = pd.concat([testeE,newF_test.T],axis=1)

a,b = iv_woe(ff, target_name, bins=10, show_woe=False)

# deleting all IVs below 0.05
a = a.loc[a['IV']> 0.05]

allNames = a.sort_values(by='IV',ascending=False).Variable.values
crossNames = [x for x in allNames if '_mul_' in x]
logCrossNames = [x for x in allNames if '_log_mul_' in x]

nomes2 = list(trainE)+crossNames+logCrossNames
nomes2.remove('Class')

In [394]:
# threshold to correlation features
threshold = 0.3

cc = ff[nomes2[2:]].corr()

mat_x = abs(cc)>threshold
mat_x = mat_x.to_numpy()

In [395]:
# selection tof the variables with low correlation, there are +- 70 features with low correlation
var1 = []
nomes = list(cc)
var1.append(nomes[0])
max_vars = 100

count = 1
for n in range(1,len(nomes)):
    
    if (mat_x[n,:n+1].sum() ) == 1:
        
        var1.append(nomes[n])        
        count+=1
        
        if(count == max_vars):
            break

In [396]:
var1 

['GL',
 'CR',
 'DA',
 'AF',
 'EE',
 'FR',
 'DE',
 'CC',
 'BN',
 'FI',
 'FE',
 'DH',
 'EU',
 'GH',
 'DY',
 'CB',
 'FC',
 'BR',
 'FS',
 'AZ',
 'CW ',
 'EG',
 'DU_mul_GL',
 'DN_mul_FE',
 'BN_mul_FS']

In [397]:
# create dic with WoE transformation
list_dics = []

for var in var1:
  df_temp = b.loc[b['Variable']==var].reset_index()
  # creating dictionary
  dict_var = {}
  for x in range(len(df_temp)):
    line = df_temp.iloc[x]
    dict_var[ line['Cutoff'] ] = line['WoE']
  list_dics.append(dict_var)

In [398]:
# train and test data
df_original = ff[var1+[target_name]+ ['k'] ]
df_teste2 = ff_teste[var1]
names = var1

In [399]:
# pandas is giving multiple warnings and making the code "dirty"
import warnings
warnings.filterwarnings('ignore')

In [400]:
# In this part there is some data leakage as the map is using the full dataset
n= 0

for var in var1:
  df_original.loc[:,var] = df_original[var].map(list_dics[n])
  df_teste2.loc[:,var] = df_teste2[var].map(list_dics[n])
  
  n=n+1

In [401]:
df_original.loc[:,names] = df_original[names].fillna(df_original[names].mean())
df_teste2.loc[:,names] = df_teste2[names].fillna(df_original[names].mean())

In [402]:
#https://www.kaggle.com/competitions/icr-identify-age-related-conditions/discussion/409691
from sklearn.utils import class_weight

def balanced_log_loss(y_true, y_pred):
    class_weights = class_weight.compute_class_weight('balanced', classes = np.unique(y_true), y = y_true)
    weights = class_weights[y_true.astype(int)]
    loss = log_loss(y_true, y_pred, sample_weight=weights)
    return loss

In [403]:
n_splits = 10

predictions_LR = 0
cv_score_LR = 0
train_score_LR = 0

rr = [42,21,100,45,1]
for v_fold in rr:
    print(v_fold)
    skf = StratifiedKFold(n_splits=n_splits,shuffle=True, random_state=v_fold)
    for i, (train_index, test_index) in enumerate(skf.split(df_original[target_name], df_original['k'])):

            model = LogisticRegression(random_state=0,C=0.1,n_jobs=-1,max_iter=2000,class_weight='balanced')

            df_train = df_original.iloc[train_index]
            df_test = df_original.iloc[test_index]
            
            
            df_train1 = df_train[names].to_numpy()
            df_test1 = df_test[names].to_numpy()            
            
            
            model.fit( df_train1, df_train[target_name])

            y_hat_teste_LR = model.predict_proba(df_test1)
            cv_score_LR += balanced_log_loss(df_test[target_name],y_hat_teste_LR)       


            y_hat_train_LR = model.predict_proba(df_train1) 
            train_score_LR += balanced_log_loss(df_train[target_name],y_hat_train_LR)  
            predictions_LR += model.predict_proba(df_teste2[names])
                  

         
print('Train :', train_score_LR/(10*len(rr)) )
print('CV score: ', cv_score_LR/(10*len(rr)) )
predictions_LR= predictions_LR/(10*len(rr))

42
21
100
45
1
Train : 0.26860645888140344
CV score:  0.3025700323789558
